# Realizando ingeniería de datos para realizar transformación y generación de nuevas variables

## 1.- Importamos las librerías necesarias y preparamos las funciones oportunas.

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re
import spacy
from nltk.stem.snowball import SnowballStemmer

nlp = spacy.load('es_core_news_lg')

In [2]:
pd.set_option('display.max_colwidth', None)       # Visualizar todas las columnas
pd.set_option('display.max_rows', None)           # Visualizar todas las filas

Generamos funciones para el tratamiento de textos

In [2]:
def extraer_lemas(texto):
    '''
    Función que recibe un texto como entrada y devuelve una lista de los lemas de las palabras que se encuentran en el texto. 
    
    La lista de lemas resultante solo contiene las palabras que son alfabéticas, es decir, que no contienen caracteres numéricos o especiales.

    - Inputs:

        texto (str): Texto del cual se quieren extraer los lemas.

    - Outputs:

        lemas (list): Lista de los lemas de las palabras alfabéticas que se encuentran en el texto.
    '''
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if token.is_alpha]
    return lemas

def eliminacion_acentos(text):
    '''
    Función que recibe un texto como entrada y devuelve el mismo texto pero sin los caracteres acentuados. 
    
    Para esto, se utiliza una expresión regular y un diccionario con los caracteres a reemplazar.

    - Inputs:

       text (str): Texto del cual se quieren eliminar los acentos.

    - Outputs:

        texto (str): Texto sin los caracteres acentuados.
         '''
    pattern = '[áéíóúÁÉÍÓÚ]'
    replace = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
    return re.sub(pattern, lambda match: replace[match.group()], text)

def spanish_stemmer(x):
    '''
    Función que recibe una cadena de texto como entrada y devuelve el texto procesado con un algoritmo de stemming en español. 
    
    El stemming es una técnica de NLP que busca reducir una palabra a su raíz. 
    
    En este caso, se utiliza el algoritmo SnowballStemmer de la librería NLTK.

    - Inputs:

        x (str): Texto que se quiere procesar con el algoritmo de stemming.

    - Outputs:

        str: Texto procesado con el algoritmo de stemming.
        '''
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

def pickled_list(lista,nombre_archivo, carpeta="Utils"):
    '''Generar fichero picke a partir de una lista de tareas. 
    
    Los ficheros se guardan en la carpeta Utils.
    
    - INPUT: 
        - lista (list):         Nombre de la lista a guardar.

        - nombre_archivo (str): Nombre del fichero a generar

        - carpeta: Carpeta donde se va a guardar el fichero .pickle

    - OUTPUT: 
        - lista (.pickle): fichero .pickle donde esta almacenada la información de la lista.

    '''
    ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")

    with open(ruta_archivo, "wb") as archivo_pickle:
            pickle.dump(lista, archivo_pickle)
    return lista

def unpickled_list(lista,nombre_archivo, carpeta="Utils"):
    '''Generar lista de tareas que existen en la carpeta Utils.
    
    Definir tareas nuevas a agregar dentro de la lista en el caso de haber una nueva tarea a añadir.
    
    - INPUT: 
        - lista (list):         Nombre de la lista a guardar.

        - nombre_archivo (str): Nombre del fichero a generar

        - carpeta:              Carpeta donde se va a guardar el fichero .pickle
    
    - OUTPUT: 
        - lista (list):         Lista de productos
    '''
    
    ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")
    with open(ruta_archivo, "rb") as archivo_pickle:
        lista = pickle.load(archivo_pickle)

    return lista





### Obtenemos los datos guardados.

In [3]:
path=r'Data\productos_scrape.csv'

In [4]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online'

In [5]:
new_path=folder+'\\'+path
new_path

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\Data\\productos_scrape.csv'

Vamos a eliminar duplicados del dataframe (esto es una operación a realizar en el anterior Notebook)

In [15]:
dataframe=pd.read_csv(new_path)
dataframe.head()

,ID,NAME,INFO,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,"Algunos lubricantes son un poco densos, otros ...",https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,"FOXTAIL, plug anal cola de zorro de 35cm",Deja volar la imaginación y saca tu lado más s...,https://www.amantis.net/foxtail-plug-anal-cola...,29.99,9.99
2,2,"LIZO 2, Dildo de suave silicona en 3 tamaños","En un azulejo de la cocina, en una puerta, en ...",https://www.amantis.net/lizo-2-dildo-suave-sil...,59.99,17.99
3,3,"Bacanal FORTE TARRO 200ml, lubricante anal con...",Si alguna vez has pensado que el sexo anal es ...,https://www.amantis.net/bacanal-forte-tarro-20...,24.99,24.99
4,4,Vibrador Líquido con sabor Desliz! VIBRAGEL 30...,¿Alguna vez has sentido un hormigueo en lo más...,https://www.amantis.net/desliz-vibragel-liquid...,29.99,9.99


In [7]:
len(dataframe)

551

## 2.- Descomponemos las variables *Name* y *Description*.

Vamos a crear 2 nuevas columnas a partir de *Name* y *Description*, donde dejaremos el nombre del producto y su slogan por un lado y por otro la descripción y las características por otro.

En principio son variables que no necesitaremos para estos estudios pero las guardaremos por si hay que hacer algún estudio posterior de las mismas (comparativas de productos similares de esta página web o de otras páginas web, por ejemplo).

#### Generación de slogan.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos el Name en una nueva columna y varias columnas para ir separando el resto. 

In [16]:
dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)
dataframe['PRODUCT'] = dataframe['PRODUCT'].str.strip()
dataframe['SLOGAN'] = dataframe['SLOGAN'].str.strip()
# dataframe.head()

C:\Users\Javier\AppData\Local\Temp\ipykernel_15480\572804979.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['NAME'] = dataframe['NAME'].str.replace(r'-(?=\w)', '_')
C:\Users\Javier\AppData\Local\Temp\ipykernel_15480\572804979.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe[['PRODUCT', 'SLOGAN']] = dataframe['NAME'].str.split('[,-.]', 1, expand=True)


#### Separación entre Description y Characteristics.

Primeramente vamos a realizar un visionado de los datos que tienen para discernir donde se puede realizar la separación.

Por un lado extraeremos las características en una nueva columna, dejando la Description en la misma. 

**Pendiente**: Hay que hacer una segunda separación de Description en el caso que solo aparezca *características*

In [18]:
dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
# dataframe.head()


C:\Users\Javier\AppData\Local\Temp\ipykernel_15480\1793188304.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['CHARACTERISTICS'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
C:\Users\Javier\AppData\Local\Temp\ipykernel_15480\1793188304.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dataframe['DESCRIPTION'] = dataframe['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()


Se observan que hay *\r* dentro del texto generado en las nuevas columnas, por lo que vamos a proceder a sustituir esto por un espacio.

In [19]:
dataframe['CHARACTERISTICS'] = dataframe['CHARACTERISTICS'].str.replace('\r', ' ')
dataframe['DESCRIPTION'] = dataframe['DESCRIPTION'].str.replace('\r', ' ')
# dataframe.head()

In [20]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               551 non-null    int64  
 1   NAME             551 non-null    object 
 2   INFO             551 non-null    object 
 3   LISTA_URL        551 non-null    object 
 4   REGULAR_PRICE    551 non-null    float64
 5   DISCOUNT_PRICE   551 non-null    float64
 6   PRODUCT          551 non-null    object 
 7   SLOGAN           455 non-null    object 
 8   CHARACTERISTICS  492 non-null    object 
 9   DESCRIPTION      551 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 43.2+ KB


Eliminamos *Nombre* y sustituimos por *Product*, *Slogan*, *Description* y *Characteristics*.

In [21]:
col_1 = dataframe.pop('PRODUCT')
col_2=dataframe.pop('SLOGAN')
col_3=dataframe.pop('DESCRIPTION')
col_4=dataframe.pop('CHARACTERISTICS')

dataframe.drop(columns=['NAME'],inplace=True)
dataframe.drop(columns=['INFO'],inplace=True)

dataframe.insert(loc= 1 , column= 'PRODUCT', value= col_1)
dataframe.insert(loc= 2 , column= 'SLOGAN', value= col_2)
dataframe.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
dataframe.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
dataframe.head()


# col_1 = dataframe.pop('Product')
# col_2=dataframe.pop('Slogan')
# col_3=dataframe.pop('Characteristics')
# dataframe.drop(columns=['Name'],inplace=True)

# dataframe.insert(loc= 0 , column= 'Product', value= col_1)
# dataframe.insert(loc= 1 , column= 'Slogan', value= col_2)
# dataframe.insert(loc= 3 , column= 'Characteristics', value= col_3)
# dataframe.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,REGULAR_PRICE,DISCOUNT_PRICE
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros ...",Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,9.99,7.99
1,1,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...,Tamaño plug S: 6.6cm Max Dia.: 2.8cmCola de pe...,https://www.amantis.net/foxtail-plug-anal-cola...,29.99,9.99
2,2,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ...",Material: silicona médica de máxima calidadDim...,https://www.amantis.net/lizo-2-dildo-suave-sil...,59.99,17.99
3,3,Bacanal FORTE TARRO 200ml,lubricante anal concentrado con aloe,Si alguna vez has pensado que el sexo anal es ...,Su textura no es grasa y no mancha. Compatible...,https://www.amantis.net/bacanal-forte-tarro-20...,24.99,24.99
4,4,Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml,hormigueo oral,¿Alguna vez has sentido un hormigueo en lo más...,"Ingredientes: agua, glicerina, hidroxietilcelu...",https://www.amantis.net/desliz-vibragel-liquid...,29.99,9.99


Pasamos a un fichero .csv para guardarlo

In [22]:
df_engineer=dataframe.iloc[:,:4]
df_engineer.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION
0,0,Desliz! Lubricante íntimo de agua 100ml,None,"Algunos lubricantes son un poco densos, otros ..."
1,1,FOXTAIL,plug anal cola de zorro de 35cm,Deja volar la imaginación y saca tu lado más s...
2,2,LIZO 2,Dildo de suave silicona en 3 tamaños,"En un azulejo de la cocina, en una puerta, en ..."
3,3,Bacanal FORTE TARRO 200ml,lubricante anal concentrado con aloe,Si alguna vez has pensado que el sexo anal es ...
4,4,Vibrador Líquido con sabor Desliz! VIBRAGEL 30ml,hormigueo oral,¿Alguna vez has sentido un hormigueo en lo más...


In [23]:
dataframe.to_csv('./Data/productos.csv',header=True,index=False)
df_engineer.to_csv('./Data/tags.csv',header=True,index=False)

## 3.- Generamos los tags para procesado de datos.

Definimos la lista de tags que queremos asignar. 

Esto lo haremos generando listas con los nombres lemmatizados.

También hay que dejar el texto utilizado en minúscula.

In [13]:
# juguetes=['dildo','plug','vibrador','masturbador','cabezal','estimulador','plugs','bolas chinas','funda extensora','bomba de vacio']
# BDSM=['latex','bdsm','arnes','strap','cera','ligadura','cuerda','cuero','sumision','dominacion','latigo','watenberg','posicionador','mordaza']
# muebles=['columpio','sillon','sillones']
# lenceria=['body','panties','corse',]
# anal=['anal']
# masculino=['hombre','masculino']
# femenino=['mujer','femenino','vaginal','clitoris','dildo']
# amenities=['lubricante','limpiador','preservativo','condon']
# posicionador=[]                         # Falta por agregar parametros 

Vamos a guardar las diferentes listas que hemos determinado en ficheros para posteriormente, cargarlas y realizar la generación de variables a partir de ellas.

In [58]:
# pickled_list(juguetes,'juguetes', carpeta="Utils")
# pickled_list(BDSM,'BDSM', carpeta="Utils")
# pickled_list(muebles,'muebles', carpeta="Utils")
# pickled_list(lenceria,'lenceria', carpeta="Utils")
# pickled_list(anal,'anal', carpeta="Utils")
# pickled_list(masculino,'masculino', carpeta="Utils")
# pickled_list(femenino,'femenino', carpeta="Utils")
# pickled_list(amenities,'amenities', carpeta="Utils")
# pickled_list(lista,nombre_archivo, carpeta="Utils")
# pickled_list(lista,nombre_archivo, carpeta="Utils")

['lubricante', 'limpiador', 'preservativo', 'condon']

In [ ]:
df_engineer=dataframe.iloc[:,:3]
df_engineer.head()

#### Pasamos todos los caracteres a minúsculas y eliminamos los acentos.

In [ ]:
df_engineer['Description'] = df_engineer['Description'].str.lower()
df_engineer['Description'] = df_engineer['Description'].apply(eliminacion_acentos)
df_engineer.head(2)

Vamos a ver como cargamos cada fichero para proceder a la generación de *tags*.


In [62]:
def unpickled_list(nombre_archivo, carpeta="Utils"):
    '''Generar lista de tareas que existen en la carpeta Utils.
    
    Definir tareas nuevas a agregar dentro de la lista en el caso de haber una nueva tarea a añadir.
    
    - INPUT: 
        - lista (list):         Nombre de la lista a guardar.

        - nombre_archivo (str): Nombre del fichero a generar

        - carpeta:              Carpeta donde se va a guardar el fichero .pickle
    
    - OUTPUT: 
        - lista (list):         Lista de productos
    '''
    
    ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")
    with open(ruta_archivo, "rb") as archivo_pickle:
        lista = pickle.load(archivo_pickle)

    return lista

In [64]:
nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']
def cargar_listas_desde_pickles(nombres_archivos, carpeta="Utils"):
    listas = {}

    for nombre_archivo in nombres_archivos:
        ruta_archivo = os.path.join(carpeta, nombre_archivo + ".pickle")

        with open(ruta_archivo, "rb") as archivo_pickle:
            lista = pickle.load(archivo_pickle)

        # Cierra el archivo
        archivo_pickle.close()

        # Almacena la lista en el diccionario
        listas[nombre_archivo] = lista

    # Devuelve el diccionario de listas
    return listas


In [66]:
listas = cargar_listas_desde_pickles(nombre_listas)

In [71]:
def aplicar_funcion_a_columna(df, nombre_lista, columna="Description"):
    # Carga la lista desde el diccionario
    lista = listas[nombre_lista]

    # Aplica la función a la columna del DataFrame
    df[nombre_lista] = df[columna].apply(lambda x: any(lematizado in lista for lematizado in extraer_lemas(x)))

    return df


In [74]:
for nombre_lista in listas:
    df_engineer = aplicar_funcion_a_columna(df_engineer, nombre_lista)

In [ ]:
df_engineer.head()

In [67]:
listas

{'amenities': ['lubricante', 'limpiador', 'preservativo', 'condon'],
 'anal': ['anal'],
 'BDSM': ['latex',
  'bdsm',
  'arnes',
  'strap',
  'cera',
  'ligadura',
  'cuerda',
  'cuero',
  'sumision',
  'dominacion',
  'latigo',
  'watenberg',
  'posicionador',
  'mordaza'],
 'femenino': ['mujer', 'femenino', 'vaginal', 'clitoris', 'dildo'],
 'masculino': ['hombre', 'masculino'],
 'juguetes': ['dildo',
  'plug',
  'vibrador',
  'masturbador',
  'cabezal',
  'estimulador',
  'plugs',
  'bolas chinas',
  'funda extensora',
  'bomba de vacio'],
 'lenceria': ['babydoll',
  'body',
  'camiseta',
  'corse',
  'corset',
  'diadema',
  'encaje',
  'falda',
  'joyeria',
  'lenceria',
  'lencero',
  'malla',
  'media',
  'panties',
  'pantis',
  'ropa',
  'sujetador',
  'vestido'],
 'muebles': ['chaise', 'columpio', 'moqueta', 'silla', 'sillon', 'sillones']}

In [63]:
juguetes=unpickled_list('juguetes', carpeta="Utils")
juguetes

['dildo',
 'plug',
 'vibrador',
 'masturbador',
 'cabezal',
 'estimulador',
 'plugs',
 'bolas chinas',
 'funda extensora',
 'bomba de vacio']

Generamos las variables nuevas a partir de las listas con las keywords.

In [58]:
# df_engineer['lenceria'] = df_engineer['Description'].apply(lambda x: any(lematizado in lenceria for lematizado in extraer_lemas(x)))
# df_engineer['juguetes'] = df_engineer['Description'].apply(lambda x: any(lematizado in juguetes for lematizado in extraer_lemas(x)))
# df_engineer['BDSM'] = df_engineer['Description'].apply(lambda x: any(lematizado in BDSM for lematizado in extraer_lemas(x)))
# df_engineer['muebles'] = df_engineer['Description'].apply(lambda x: any(lematizado in muebles for lematizado in extraer_lemas(x)))
# df_engineer['anal'] = df_engineer['Description'].apply(lambda x: any(lematizado in anal for lematizado in extraer_lemas(x)))
# df_engineer['masculino'] = df_engineer['Description'].apply(lambda x: any(lematizado in masculino for lematizado in extraer_lemas(x)))
# df_engineer['femenino'] = df_engineer['Description'].apply(lambda x: any(lematizado in femenino for lematizado in extraer_lemas(x)))
# df_engineer['amenities'] = df_engineer['Description'].apply(lambda x: any(lematizado in amenities for lematizado in extraer_lemas(x)))

# df_engineer['otros'] = ~(df_engineer['lenceria'] | df_engineer['juguetes'] | df_engineer['BDSM']| df_engineer['muebles'] | df_engineer['anal']| df_engineer['masculino'] | df_engineer['femenino'] | df_engineer['amenities']).any()


# df_engineer['Description'] = df_engineer['Description'].apply(spanish_stemmer)

# df_engineer

Hacemos una visualización de las cantidades que hay en cada una de las nuevas columnas para después ver si han realizado bien la separación o no y volver a definirlas.

Haremos después una visualización uno a uno de los diferentes tags generados para verificar si están bien definidos y en el caso que haya que realizar una modificación generar una nueva lista para incluirla en las iniciales.

##### Lencería

In [28]:
df_engineer['lenceria'].value_counts()

False    525
True      32
Name: lenceria, dtype: int64

In [ ]:
df_engineer[df_engineer['lenceria']==True]

Hay vibradores dentro de esta lista porque en el texto está la palabra body.

In [ ]:
df_engineer[df_engineer['lenceria']==False]

In [55]:
lenceria_new=['vestido','media','falda','ropa','lenceria','lencero','malla','sujetador','corset','camiseta','pantis','encaje','babydoll','diadema','joyeria']
lenceria+=lenceria_new
lenceria.sort()
lenceria

['babydoll',
 'body',
 'camiseta',
 'corse',
 'corset',
 'diadema',
 'encaje',
 'falda',
 'joyeria',
 'lenceria',
 'lencero',
 'malla',
 'media',
 'panties',
 'pantis',
 'ropa',
 'sujetador',
 'vestido']

In [30]:
df_engineer['lenceria'] = df_engineer['Description'].apply(lambda x: any(lematizado in lenceria for lematizado in extraer_lemas(x)))

In [31]:
df_engineer['lenceria'].value_counts()

False    462
True      95
Name: lenceria, dtype: int64

In [ ]:
df_engineer[df_engineer['lenceria']==True]

Hay varios productos que no deben de estar, tienen vibrador, succionador y mordaza.

Estos productos deberían de cambiar de True a False.

In [ ]:
df_engineer[df_engineer['lenceria']==False]

Qué hacemos con los collares y las cadenas para pezones?

##### Juguetes

In [32]:
df_engineer['juguetes'].value_counts()

False    311
True     246
Name: juguetes, dtype: int64

In [ ]:
df_engineer[df_engineer['juguetes']==True]

En este punto se han incluido como juguetes productos que no lo son como amenities (limpiadores, para guardar) o arneses.

In [ ]:
df_engineer[df_engineer['juguetes']==False]

Se ve que hay bastantes productos mal asignados, principalmente anillas para penes, bombas de succion, hay plugs que no se han asignado bien, bolas chinas,...

##### BDSM

In [33]:
df_engineer['BDSM'].value_counts()

False    411
True     146
Name: BDSM, dtype: int64

In [ ]:
df_engineer[df_engineer['BDSM']==True]

In [ ]:
df_engineer[df_engineer['BDSM']==False]

##### Muebles

In [34]:
df_engineer['muebles'].value_counts()

False    552
True       5
Name: muebles, dtype: int64

In [ ]:
df_engineer[df_engineer['muebles']==True]

Hay que cambiar Black Line

In [ ]:
df_engineer[df_engineer['muebles']==False]

In [56]:
muebles_new=['chaise','silla','moqueta']          # Pendiente de agregar, no puedo usar puerta
muebles+=muebles_new
muebles.sort()
muebles

['chaise', 'columpio', 'moqueta', 'silla', 'sillon', 'sillones']

Agregando los nuevos valores

In [36]:
df_engineer['muebles'] = df_engineer['Description'].apply(lambda x: any(lematizado in muebles for lematizado in extraer_lemas(x)))
df_engineer['muebles'].value_counts()

False    543
True      14
Name: muebles, dtype: int64

In [ ]:
df_engineer[df_engineer['muebles']==True]

Hay que eliminar los dildos y plugs y agregar el soporte para puertas

##### Anal

In [24]:
df_engineer['anal'].value_counts()

False    453
True     123
Name: anal, dtype: int64

In [ ]:
df_engineer[df_engineer['anal']==True]

In [ ]:
df_engineer[df_engineer['anal']==False]

##### Masculino

In [27]:
df_engineer['masculino'].value_counts()

False    525
True      51
Name: masculino, dtype: int64

In [ ]:
df_engineer[df_engineer['masculino']==True]

In [ ]:
df_engineer[df_engineer['masculino']==False]

##### Femenino

In [59]:
df_engineer['femenino'].value_counts()

False    376
True     200
Name: femenino, dtype: int64

In [ ]:
df_engineer[df_engineer['femenino']==True]

Hay juguetes masculinos que se han incluido aquí dado que tienen la palabra mujer. 

Estos son las muñecas para hombres. 

In [ ]:
df_engineer[df_engineer['femenino']==False]

##### Amenities

In [37]:
df_engineer['amenities'].value_counts()

False    389
True     168
Name: amenities, dtype: int64

In [ ]:
amenities=['lubricante','limpiador','preservativo','condon']            #Tengo que aplicar primero la generación de las variables

In [ ]:
df_engineer[df_engineer['amenities']==True]

In [ ]:
df_engineer[df_engineer['amenities']==False]

##### Otros

In [31]:
df_engineer['otros'].value_counts()

False    576
Name: otros, dtype: int64

In [37]:
df_engineer[df_engineer['otros']==True]

,Name,Description,lenceria,juguetes,BDSM,muebles,anal,masculino,femenino,amenities,otros


In [ ]:
df_engineer[df_engineer['otros']==False]

#### Generación de tags.

Vamos a tratar un texto para ver como se puede generar tags de los mismos. Para ello vamos a cargar la librería *spaCy* y tratar un texto para tokenizar y demás ...

In [ ]:
text = """
¿Encuentras placer en que te dominen? ¿Sientes que dominas tu desde la sumisión? Todo este juego de roles es altamente excitante, y desde amantis te invitamos a que lo explores.

Una buena manera de empezar, y uno de los símbolos más conocidos en la sumisión son los collares, digamos que sería el equivalente al anillo de compromiso cuando te casas. De hecho, existe una ceremonia llamada la coronización, en donde la persona sumisa acepta estar en una relación de confianza, estabilidad, lealtad y protección.

YOURS es un collar perteneciente a la categoría de sumisión de BDSM, y podremos usarlo tanto en público como en nuestras sesiones privadas con nuestro amo o ama. Tiene una potente hebilla con ocho agujeros, por lo que acepta todo tipo de tamaño de cuellos. A su vez, YOURS tiene un amarre en el centro por si quisiéramos llevarlo más allá de la simbología y quisieramos atarle una correa para que las persona que nos dirige sea dueña de nuestro movimientos.

A diferencia de otros collares tradicionalmente negros o rojos y de estética fetiche, este collar opta por otros colores abriendo más la gama de esta categoria. Además, YOURS apuesta por la comodidad, y el material del que está hecho es blando, estilo neopreno, que será más amable con nuestra piel permitiéndonos llevarlo más tiempo sin tener rozaduras.

Cada pieza tiene hebillas y agarres plateadas, y los que son amarres vienen con zona de agujeros que harán que puedas ajustarlas a la medidas que quieras.

Ten cuenta principio fundamental del BDSM, el SCCP (sexo seguro, sensato, consensuado y placentero) y no olvideis tener una palabra de seguridad que nos ayude a entender si algún límite está siendo sobrepasado."""

doc = nlp(text)

print(doc)

for num, sentence in enumerate(doc.sents):
  print(num, sentence)
